Define annotations for each cell subtype that capture the set of peaks that best reflect that cell subtype (ie whose accessibility profiles best track with that cell subtype's PC values)

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy.stats import norm
import anndata as adata
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *
from regression_dependencies import *

### Defining peak-cell subtype correlations, as input to bed for annotation/LDSC/h2 input

In [ ]:
ct='T'
# get cell subtype mappings
ct_adata = sc.read_h5ad("/data/srlab/agupta/data/d12_matrices/processed_h5ads/"+ct+"_RNA_adata.h5ad")

#### subtype-based

In [ ]:
#cell-PC loadings
res = pd.read_csv("/data/srlab/agupta/data/d12_matrices/cellPC-loadings/"+ct+"_RNA_PC_loadings.csv",index_col=0)
res = res.drop(['PC'+str(i) for i in np.arange(11,21,1)]+['donor','donor_num','total_counts'],axis=1)
res['subtype'] = list(ct_adata[res.index].obs['ct_subtype'])

ALL_PC_cell_scores = pd.DataFrame()
all_annots = ['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10']
for PC in all_annots:
    max_val = 1
    cell_PC_loadings = np.array([i/max_val for i in res[PC]])
    PC_cell_scores_df = pd.DataFrame(cell_PC_loadings)
    PC_cell_scores_df.columns = [PC]
    PC_cell_scores_df.index = res.index
    ALL_PC_cell_scores = pd.concat([ALL_PC_cell_scores,PC_cell_scores_df],axis=1)

ALL_PC_cell_scores['subtype'] = res['subtype']

#mean loading for all cells in subtype
cell_subtype_PC_scores = ALL_PC_cell_scores.groupby('subtype').mean()

#### peaks x PCs info

In [ ]:
#peak Z-scores for the 10 PC categories
OG_df = pd.DataFrame(pd.read_csv('/data/srlab/agupta/data/peak_gene_scores/d12_scores/'+ct+'/083122_poisson_regression_10PC_all_outputs.csv',index_col=0))
#only include DYNAMIC peaks for this peak to cell correlation mapping
peaks_CT_reg_df = OG_df[OG_df['model_LRT_qval']<0.05]
peaks_CT_reg_df = peaks_CT_reg_df[['PC1 zscore','PC2 zscore','PC3 zscore','PC4 zscore','PC5 zscore', 'PC6 zscore','PC7 zscore','PC8 zscore','PC9 zscore','PC10 zscore']]

ALL_PC_PEAK_scores = pd.DataFrame()
all_annots = ['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10']
for annot in all_annots:
    PEAK_PC_zscores = peaks_CT_reg_df[annot+' zscore']
    PC_PEAK_scores_df = pd.DataFrame(PEAK_PC_zscores)
    PC_PEAK_scores_df.columns = [annot]
    PC_PEAK_scores_df.index = peaks_CT_reg_df.index
    ALL_PC_PEAK_scores = pd.concat([ALL_PC_PEAK_scores,PC_PEAK_scores_df],axis=1)

#### cosine similarity

In [ ]:
## FULL COSINE SIMILARITY DF ##
from sklearn.metrics.pairwise import cosine_similarity

cell_PC_df_use = cell_subtype_PC_scores

num_peaks = len(ALL_PC_PEAK_scores)
chunk_size = int(num_peaks/20)
cell_peak_corrs_for_annots = pd.DataFrame()
for i in np.arange(chunk_size,num_peaks,chunk_size):
    #subset peaks to this chunk
    peak_subset_df = pd.concat([cell_PC_df_use,ALL_PC_PEAK_scores.iloc[i-chunk_size:i,:]],axis=0)
    #calculate peak-cell corrs for this  peak set/chunk
    #correlation for each "avg" cell - peak pair (across the 10 PC dimensions)
    print('calculating cosine similarity-based corrs for peak set', i-chunk_size, i)
    subset_cosine_corrs = pd.DataFrame(cosine_similarity(peak_subset_df)) # cosine similarity
    subset_cosine_corrs.index = list(cell_PC_df_use.index) + list(ALL_PC_PEAK_scores.iloc[i-chunk_size:i,:].index)
    subset_cosine_corrs.columns = list(cell_PC_df_use.index) + list(ALL_PC_PEAK_scores.iloc[i-chunk_size:i,:].index)
    subset_cosine_corrs = subset_cosine_corrs.iloc[:len(cell_PC_df_use), len(cell_PC_df_use):].T #num cell subtypes
    
    cell_peak_corrs_for_annots = pd.concat([cell_peak_corrs_for_annots, subset_cosine_corrs])

In [ ]:
#to see how the pvals are distributed for each cell subtype
for col in cell_peak_corrs_for_annots.columns:
    plt.figure(figsize=(4,3))
    plt.hist(cell_peak_corrs_for_annots[col],bins=50);
    plt.title(col)
    sns.despine()
    plt.show()

In [ ]:
#convert to BED score
print('copying cell-peak corrs')
peak_scores = cell_peak_corrs_for_annots.copy()
print('excluding chrX and Y')
peak_scores = peak_scores[peak_scores.index.str.contains('chrX')==False]
peak_scores = peak_scores[peak_scores.index.str.contains('chrY')==False]
# if T cells:
# peak_scores = peak_scores.drop('T-19: MT-high (low quality)', axis=1)
# peak_scores.columns = [i.split(":")[0] for i in peak_scores.columns]
peak_scores

In [ ]:
# binarize scores (top 10% of peaks = 1; rest = 0)
print("ranking and subsetting peaks")
peak_scores = peak_scores.where(peak_scores.rank(axis=0,ascending=False)<(len(peak_scores)*(0.1))).fillna(0) #CHANGE BASED ON THRESHOLD TO USE FOR "TOP" PEAKS FOR EACH CELL SET

for st in peak_scores.columns:
    plt.figure(figsize=(4,3))
    plt.hist(peak_scores[st],bins=30);
    plt.title(st)
    sns.despine()
    plt.show()
    
print('binarizing peak scores')
#binarize scores
peak_scores[peak_scores>0]=1

In [ ]:
#convert values to int to make calcs faster (and can do this now since we've binarized)
peak_scores = peak_scores.astype(int)
#num peaks in each pseudocell's annotation
print(np.sum(peak_scores))
#make DF sparse to output as csv (for storage and future reference)
peak_scores_binarized_sparse = peak_scores.astype(pd.SparseDtype("int", np.nan))

#### convert to hg38 bed files and output

In [ ]:
# ACTUALLY CONVERT TO INDIVIDUAL .BED FILES
full_df = peak_scores_binarized_sparse
peak_window = 500

chromosome = [str(i).split(":")[0] for i in full_df.index]
peak_start_bp = [str(i).split(":")[1].split("-")[0] for i in full_df.index]
peak_end_bp = [str(i).split(":")[1].split("-")[1] for i in full_df.index]

peaks_for_bed_df = pd.DataFrame([chromosome, peak_start_bp, peak_end_bp]).T
peaks_for_bed_df.columns = ['chr','start','end']

i=0
for annot_type in full_df.columns:
    i+=1
    if i%2==0: #print status update at every 500th cell
        print(annot_type)

    peaks_for_bed_df['SCORE'] = full_df[annot_type].values
    
    # #to make the window around peaks wider
    peaks_for_bed_df_extended = peaks_for_bed_df.copy()
    peaks_for_bed_df_extended['start'] = [int(i) - peak_window for i in list(peaks_for_bed_df_extended['start'])]
    peaks_for_bed_df_extended['end'] = [int(i) + peak_window for i in list(peaks_for_bed_df_extended['end'])]
    # print(peaks_for_bed_df_extended.head())
    print(np.sum(peaks_for_bed_df_extended['SCORE']))
    # #output without index or header
    bed_folder = '/data/srlab/agupta/data/h2/bed/d12/subtypes_hg38/'
    peaks_for_bed_df_extended.to_csv(bed_folder+"083122_"+annot_type+"_binary_hg38.bed",
                                     header=False,
                                     index=False,
                                     sep='\t')